In [2]:
pip install pandas numpy scikit-learn geopandas shapely fiona pyproj rtree xgboost


  Using cached geopandas-1.1.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached rtree-1.4.1-py3-none-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached click_plugins-1.1.1.2-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
Using cached geopandas-1.1.1-py3-none-any.whl (338 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 48.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 32.9 MB/s eta 0:00:00
Using cached rtree-1.4.1-py3-none-macosx_11_0_arm64.whl (436 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.3 MB/s eta 0:00:00
Using cached click_plugins-1.1.1.2-py2.py3-none-any.whl (11 kB)
Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 45.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [geopandas]/9 [geopandas]ins]


In [3]:
pip install dbfread

  Using cached dbfread-2.0.7-py2.py3-none-any.whl.metadata (3.3 kB)
Using cached dbfread-2.0.7-py2.py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import warnings
from datetime import timedelta
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor
import geopandas as gpd
from shapely.geometry import Point

warnings.filterwarnings("ignore")

# ============================================================
# 1. LOAD DATA
# ============================================================

def load_raw_data():
    """
    Load the NYC Foursquare check-in dataset.
    """
    file_path = "/Users/chardog234/Documents/GitHub/Final_project_4003/dataset_TSMC2014_NYC.csv"
    df = pd.read_csv(file_path, encoding="ISO-8859-1")
    print(f"✅ Loaded dataset: {len(df)} samples")
    return df


# ============================================================
# 2. NORMALIZE TIMES
# ============================================================

def normalize_times(df):
    """
    Convert UTC timestamps to local time and extract:
      - Day_of_Week
      - Hour (0–23)
    """
    df = df.copy()

    df["UTC_Time"] = pd.to_datetime(
        df["utcTimestamp"],
        utc=True,
        format="%a %b %d %H:%M:%S +0000 %Y",
        errors="coerce"
    )

    # Drop invalid timestamps
    df = df.dropna(subset=["UTC_Time"])

    # Convert to local time using timezoneOffset (in minutes)
    df["Local_Time"] = df["UTC_Time"] - pd.to_timedelta(df["timezoneOffset"], unit="m")
    df["Day_of_Week"] = df["Local_Time"].dt.day_name()
    df["Hour"] = df["Local_Time"].dt.hour

    return df


# ============================================================
# 3. ATTACH BOROUGH SHAPES
# ============================================================

def attach_borough_zones(df):
    """
    Spatially join check-ins to real NYC borough polygons.
    Adds a 'Borough_ID' column containing the borough name.
    """
    print("📍 Attaching REAL NYC borough boundaries...")

    shp = "/Users/chardog234/Documents/GitHub/Final_project_4003/geo_export_3c9604c8-be29-4e21-ae16-9c7d64ebce63.shp"
    boroughs = gpd.read_file(shp).to_crs(epsg=4326)

    BOROUGH_COL = "boroname"  # confirmed from DBF

    # Convert (longitude, latitude) to geometry points
    gdf = gpd.GeoDataFrame(
        df.copy(),
        geometry=[Point(xy) for xy in zip(df["longitude"], df["latitude"])],
        crs="EPSG:4326"
    )

    # Spatial join: each check-in gets assigned a borough
    gdf_join = gpd.sjoin(
        gdf,
        boroughs[[BOROUGH_COL, "geometry"]],
        how="left",
        predicate="within"
    )

    gdf_join = gdf_join.rename(columns={BOROUGH_COL: "Borough_ID"})
    gdf_join = gdf_join.dropna(subset=["Borough_ID"])
    gdf_join = gdf_join.drop(columns=["geometry", "index_right"], errors="ignore")

    print(f"🏙️ Matched: {len(gdf_join)} check-ins to boroughs")
    return gdf_join


# ============================================================
# 4. BUILD SOFT-LABEL CONTEXTS
# ============================================================

def build_soft_contexts(df):
    """
    Build soft-label probability distributions for each context:
      Context = (Borough_ID, Day_of_Week, Hour)

    For each context, compute:
      P(venueCategory | borough, day, hour) = freq / total freq in that context

    Returns a pivoted DataFrame with:
      - columns: [Borough_ID, Day_of_Week, Hour, <venueCategory1>, <venueCategory2>, ...]
      - each venueCategory column = probability in that context
    """
    print("📊 Building soft-label distributions...")

    group_cols = ["Borough_ID", "Day_of_Week", "Hour"]

    # Count check-ins per (borough, day, hour, venueCategory)
    counts = (
        df.groupby(group_cols + ["venueCategory"])
          .size()
          .reset_index(name="freq")
    )

    # Normalize frequencies within each context → probabilities
    counts["prob"] = counts["freq"] / counts.groupby(group_cols)["freq"].transform("sum")

    # Pivot to wide format: 1 row per context, 1 column per venue category
    df_pivot = counts.pivot_table(
        index=group_cols,
        columns="venueCategory",
        values="prob",
        fill_value=0.0
    ).reset_index()

    print(f"➡️ Created {len(df_pivot)} unique contexts with probability distributions.")
    return df_pivot


# ============================================================
# 5. MODEL (MULTI-OUTPUT REGRESSION)
# ============================================================

class MovementModel:
    """
    Multi-output regression model that predicts
    P(venueCategory | borough, day, hour)
    as a probability distribution over categories.
    """

    def __init__(self):
        self.model = None
        self.enc_borough = LabelEncoder()
        self.enc_day = LabelEncoder()
        self.enc_hour = LabelEncoder()
        self.categories = None

    # --------------------------------------------------------
    def fit(self, df_ctx):
        """
        Fit the model on context-level probability distributions.
        """
        df = df_ctx.copy()

        # Identify all venue category columns (everything except the keys)
        self.categories = [c for c in df.columns if c not in ["Borough_ID", "Day_of_Week", "Hour"]]

        # Encode inputs
        df["borough_enc"] = self.enc_borough.fit_transform(df["Borough_ID"])
        df["day_enc"] = self.enc_day.fit_transform(df["Day_of_Week"])
        df["hour_enc"] = self.enc_hour.fit_transform(df["Hour"])

        X = df[["borough_enc", "day_enc", "hour_enc"]]
        Y = df[self.categories]

        X_train, X_test, Y_train, Y_test = train_test_split(
            X, Y, test_size=0.2, random_state=42
        )

        print("🧠 Training multi-output XGBoost Regressor...")
        base = XGBRegressor(
            objective="reg:squarederror",
            n_estimators=300,
            learning_rate=0.07,
            max_depth=8,
            subsample=0.9,
            colsample_bytree=0.9
        )

        self.model = MultiOutputRegressor(base)
        self.model.fit(X_train, Y_train)

        preds = self.model.predict(X_test)
        mse = mean_squared_error(Y_test.values.flatten(), preds.flatten())
        print(f"✨ Validation MSE: {mse:.6f}")

    # --------------------------------------------------------
    def predict_distribution(self, borough, day, hour, temperature=1.5):
        """
        Predict a smoothed probability distribution over venue categories
        for a given (borough, day, hour).

        temperature > 1.0 → softer probabilities (less peaky)
        """
        X = pd.DataFrame([{
            "borough_enc": self.enc_borough.transform([borough])[0],
            "day_enc": self.enc_day.transform([day])[0],
            "hour_enc": self.enc_hour.transform([hour])[0]
        }])

        raw = self.model.predict(X)[0]

        # Clip small negatives due to regression noise
        raw = np.maximum(raw, 0)

        # Temperature smoothing to avoid extremely sharp peaks
        raw = raw ** (1.0 / temperature)

        # Re-normalize to sum to 1
        total = raw.sum()
        if total == 0:
            # Fallback: uniform distribution if model is degenerate
            probs = np.ones_like(raw) / len(raw)
        else:
            probs = raw / total

        return pd.Series(probs, index=self.categories).sort_values(ascending=False)


# ============================================================
# 6. MAIN SCRIPT (TEST RUN)
# ============================================================

def main():
    # 1. Load and preprocess data
    df = load_raw_data()
    df = normalize_times(df)
    df = attach_borough_zones(df)

    # 2. Build soft probability contexts
    df_ctx = build_soft_contexts(df)

    # 3. Train model
    model = MovementModel()
    model.fit(df_ctx)

    # 4. Example prediction
    preds = model.predict_distribution("Manhattan", "Monday", 10, temperature=1.5)
    print(preds.head(15))


if __name__ == "__main__":
    main()


✅ Loaded dataset: 227428 samples
📍 Attaching REAL NYC borough boundaries...
🏙️ Matched: 181871 check-ins to boroughs
📊 Building soft-label distributions...
➡️ Created 837 unique contexts with probability distributions.
🧠 Training multi-output XGBoost Regressor...
✨ Validation MSE: 0.000337
Bar                                         0.074189
Hotel                                       0.048733
Deli / Bodega                               0.032866
Residential Building (Apartment / Condo)    0.031649
Other Great Outdoors                        0.030818
Subway                                      0.030680
Medical Center                              0.030602
Museum                                      0.029859
German Restaurant                           0.029016
Burger Joint                                0.025291
Fast Food Restaurant                        0.024238
Home (private)                              0.023779
Latin American Restaurant                   0.023653
Movie Theater       